# Validation of Gluon Models on ImageNet

We are about to test the performance of gluon pretrained models in the [model zoo](https://github.com/apache/incubator-mxnet/tree/master/python/mxnet/gluon/model_zoo/vision) on ImageNet validation dataset. The validation dataset consists of 50,000 images, 50 per class.

To begin with, import the necessary modules.

In [1]:
import mxnet as mx
import numpy as np
from mxnet import gluon, nd
from mxnet import autograd as ag
from mxnet.gluon import nn
from mxnet.gluon.model_zoo import vision as models
from mxnet.gluon.data import vision
from mxnet.gluon.data.vision import transforms

import shutil
import time
import os
import pandas as pd

## Data Download

First we need to prepare the [validation images](http://www.image-net.org/download-images) and preprocess them. Due to the black magic of Gluon, we do not need to transform the JPEG files to `rec` format.

The DataLoader in Gluon requires the images arranged in the subdir named after the image label. However, the validation images are stored in one folder, so we need to create a subdir for every label, and put a symbol link to the image in the every subdir.

In [2]:
val_path = 'val'
image_path = '/unsullied/sharefs/luojing/exp/Dataset/ImageNet2012Val/ILSVRC2012_img_val' #use absolute address
synsets_file = open('../basics/synsets.txt', 'r')
val_file = open('../basics/val.txt', 'r')

if os.path.exists(val_path):
    shutil.rmtree(val_path)

synsets = [line.rstrip('\n') for line in synsets_file.readlines()]

for line in val_file.readlines():
    fname, idx = line.split()
    label_path = '%s/%s' % (val_path, synsets[int(idx)])
    if not os.path.exists(label_path):
        os.makedirs(label_path)
    os.symlink('%s/%s' % (image_path, fname), '%s/%s' % (label_path, fname))

The common benchmark of ImageNet validation is Top 1/5 error of `224x` on `256x` Crop. That is how we define the tranform function.

In [3]:
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

We use `ImageFolderDataset` to read the validation dataset.

In [4]:
batch_size = 25
num_gpus = 8
batch_size *= max(1, num_gpus)
val_total = 50000
ctx = [mx.gpu(i) for i in range(num_gpus)] if num_gpus > 0 else [mx.cpu()]
num_workers = 10

val_dataset = vision.ImageFolderDataset(val_path)
val_data = gluon.data.DataLoader(
        val_dataset.transform_first(transform_test),
        batch_size=batch_size, shuffle=False, num_workers=num_workers)

## Load Models and Test

In [5]:
from mxnet.gluon.model_zoo.model_store import _model_sha1

test_result = []
acc_top1 = mx.metric.Accuracy()
acc_top5 = mx.metric.TopKAccuracy(5)

for model in sorted(_model_sha1.keys()):
    if model == 'inceptionv3':
        continue
    net = models.get_model(model, pretrained=True, ctx=ctx)
    acc_top1.reset()
    acc_top5.reset()    
    for _, batch in enumerate(val_data):       
        data = gluon.utils.split_and_load(batch[0], ctx)
        label = gluon.utils.split_and_load(batch[1], ctx)       
        outputs = [net(X) for X in data]
        acc_top1.update(label, outputs)
        acc_top5.update(label, outputs)
        # print_str = 'Top 1 Err: %4f \t Top 5 Err: %4f '%(1 - top1, 1 - top5)
        # pbar.set_description("%s" % print_str)
    _, top1 = acc_top1.get()
    _, top5 = acc_top5.get()
    print('Model: %s \t Top 1 Err: %4f \t Top 5 Err: %4f '%(model, 1 - top1, 1 - top5))
    test_result.append((model, 1 - top1, 1 - top5))   
        

Model: alexnet 	 Top 1 Err: 0.470920 	 Top 5 Err: 0.233480 
Model: densenet121 	 Top 1 Err: 0.254400 	 Top 5 Err: 0.078040 
Model: densenet161 	 Top 1 Err: 0.223880 	 Top 5 Err: 0.060680 
Model: densenet169 	 Top 1 Err: 0.238380 	 Top 5 Err: 0.068240 
Model: densenet201 	 Top 1 Err: 0.231000 	 Top 5 Err: 0.065260 
Model: mobilenet0.25 	 Top 1 Err: 0.499980 	 Top 5 Err: 0.255280 
Model: mobilenet0.5 	 Top 1 Err: 0.380900 	 Top 5 Err: 0.159200 
Model: mobilenet0.75 	 Top 1 Err: 0.334320 	 Top 5 Err: 0.127500 
Model: mobilenet1.0 	 Top 1 Err: 0.297760 	 Top 5 Err: 0.103600 
Model: resnet101_v1 	 Top 1 Err: 0.233680 	 Top 5 Err: 0.066580 
Model: resnet101_v2 	 Top 1 Err: 0.227520 	 Top 5 Err: 0.063480 
Model: resnet152_v1 	 Top 1 Err: 0.230380 	 Top 5 Err: 0.064820 
Model: resnet152_v2 	 Top 1 Err: 0.219760 	 Top 5 Err: 0.060840 
Model: resnet18_v1 	 Top 1 Err: 0.336340 	 Top 5 Err: 0.126860 
Model: resnet18_v2 	 Top 1 Err: 0.310500 	 Top 5 Err: 0.113540 
Model: resnet34_v1 	 Top 1 Err: 0.

In [6]:
summary = pd.DataFrame(test_result, columns=['model', 'top_1_err', 'top_5_err'])
summary = summary.sort_values('top_1_err')
summary.head()

,model,top_1_err,top_5_err
12,resnet152_v2,0.21976,0.06084
2,densenet161,0.22388,0.06068
10,resnet101_v2,0.22752,0.06348
11,resnet152_v1,0.23038,0.06482
4,densenet201,0.23100,0.06526


In [7]:
for i, (model, top1, top5) in summary.iterrows():
    print('| %s | %.2f | %.2f |'%(model, top1 * 100, top5 * 100))

| resnet152_v2 | 21.98 | 6.08 |
| densenet161 | 22.39 | 6.07 |
| resnet101_v2 | 22.75 | 6.35 |
| resnet152_v1 | 23.04 | 6.48 |
| densenet201 | 23.10 | 6.53 |
| resnet101_v1 | 23.37 | 6.66 |
| densenet169 | 23.84 | 6.82 |
| resnet50_v2 | 24.04 | 7.19 |
| resnet50_v1 | 24.92 | 7.48 |
| densenet121 | 25.44 | 7.80 |
| resnet34_v2 | 27.44 | 8.90 |
| vgg19_bn | 28.92 | 9.66 |
| resnet34_v1 | 29.46 | 9.81 |
| vgg16_bn | 29.71 | 9.93 |
| mobilenet1.0 | 29.78 | 10.36 |
| vgg19 | 30.56 | 10.74 |
| resnet18_v2 | 31.05 | 11.35 |
| vgg16 | 31.59 | 11.31 |
| vgg11_bn | 32.70 | 12.18 |
| vgg13_bn | 32.83 | 12.01 |
| mobilenet0.75 | 33.43 | 12.75 |
| resnet18_v1 | 33.63 | 12.69 |
| vgg13 | 33.70 | 12.70 |
| vgg11 | 34.86 | 13.50 |
| mobilenet0.5 | 38.09 | 15.92 |
| squeezenet1.0 | 45.95 | 22.67 |
| alexnet | 47.09 | 23.35 |
| squeezenet1.1 | 47.78 | 23.71 |
| mobilenet0.25 | 50.00 | 25.53 |


In [8]:
summary.to_csv('model_zoo_test_result.csv', index=None)